This file intends to extract all the information from the input sheet and store it in a dictionary. The keys would be file name cell name, and the values would be the func name and the arguments, where the arguments are {arg_name: value}

In [1]:
import pandas as pd
import os
import numpy as np
import json
from collections import Counter
from collections import OrderedDict

In [2]:
# read in the data
def load_data(path):
    """ read in excel sheets
        grab all .xlsx files in the dir specified by path """
    files_under_path = os.listdir(path)
    fnames = [f[:-5] for f in files_under_path if f.endswith('.xlsx')]

    all_files = {}
    for i in range(len(fnames)):
        file = pd.read_excel(f'{path}/{fnames[i]}.xlsx', sheet_name=None, engine='openpyxl')
        all_files[fnames[i]] = file

    return fnames, all_files

In [3]:
os.getcwd()

'c:\\Users\\yihuaqia\\work\\markdraw_stat\\source_code\\dev_test\\analysis'

In [4]:
fnames, all_files = load_data('../../input_sheets/1227')

In [5]:
all_files.keys()

dict_keys(['canon_1227_d3t4_jm2_siv_2x', 'canon_1227_d3t7_jm5_siv_2x', 'canon_1227_d3t7_psv', 'canon_1227_gmb_siv_2x', 'canon_1227_gms_siv_2x', 'canon_lunarlake_psm_final_1'])

In [6]:
fnames  # list of file names

['canon_1227_d3t4_jm2_siv_2x',
 'canon_1227_d3t7_jm5_siv_2x',
 'canon_1227_d3t7_psv',
 'canon_1227_gmb_siv_2x',
 'canon_1227_gms_siv_2x',
 'canon_lunarlake_psm_final_1']

In [7]:
all_files[fnames[1]].keys()  # sheet names

dict_keys(['canon', 'all_functions', 'all_marks', 'parents', 'review_parent', 'dummy_beard', 'dummy_filler', 'fdr_dummy', 'kga', 'diag_dummy', 'zonal_bkg', 'xy4', 'pound', 'hatch', 'swirl_xy4', 'swirl_cross', 'swirl_hatch', 'marklist', 'delivery'])

In [ ]:
def get_cell_info2(file, output_file=None):
    """ get the cell names from the StartLayoutAssembler or zonal_background """
    df_list = []
    for sname in file.keys():
        cell_names = []
        df = pd.DataFrame(file[sname])
        # check if the StartLayoutAssembler is in any place in the df
        if 'StartLayoutAssembler' not in df.values and 'zonal_background' not in df.values:
            continue
        
        # get cell names; i is the row index
        for i in range(df.shape[0]):
            # print(f'{i}: {df.iloc[i,1]}')
            if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
                c_name = df.iloc[i]['CellName.string']
                if c_name not in cell_names:
                    cell_names.append((c_name,i))

        # print(f'{fname}: {cell_names}')
        # slice the cells based on cellnames
        df_cells = {}  # HERE STILL NEEDS A FIX
        for i in range(len(cell_names)-1):
            df_cells[cell_names[i][0]] = df.iloc[cell_names[i][1]:cell_names[i+1][1],:].dropna(how='all')
        df_cells[cell_names[-1][0]] = df.iloc[cell_names[-1][1]:,:].dropna(how='all')
        
        df_list.append(df_cells)
    return df_list

        



In [83]:
if not os.path.exists('./test_output'):
    os.makedirs('./test_output')
with open('./test_output/tmp_write.txt', 'w+') as test_write:
    lst = get_cell_info2(all_files['canon_1227_d3t4_jm2_siv_2x'],test_write)

In [84]:
lst[1].keys()

dict_keys(['122700c_d3t4_siv_swirl', '122700c_d3t4_jm2_dummy_3_1', '122700c_d3t4_siv022d', '122700c_d3t4_siv023d', '122700c_d3t4_siv024d', '122700c_d3t4_siv078d', '122700c_d3t4_siv173d', '122700c_d3t4_siv222d', '122700c_d3t4_siv223d', '122700c_d3t4_siv224d', '122700c_d3t4_siv355d', '122700c_d3t4_siv374d'])

In [85]:
lst[1]['122700c_d3t4_jm2_dummy_3_1']

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
16,y,StartLayoutAssembler,canon27d3t4lib1_p27_lay,122700c_d3t4_jm2_dummy_3_1,w,59.74,61.72,0,0,NaN
17,y,diagonal_gratings,cv,JM2_mask.frameDrawing,135,2.004,3.006,2.2,0,NaN


In [87]:
lst[1]['122700c_d3t4_siv355d']

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
62,y,StartLayoutAssembler,canon27d3t4lib2_p27_lay,122700c_d3t4_siv355d,a,61.74,63.72,0,0,NaN
63,y,create_rectangle,cv,or2.tccDebug,-24 -24 24 24,NaN,NaN,NaN,NaN,NaN
64,y,tvpa_canon,cv,JM2_mask.drawing,2.2,33.8,33.8,33.8,NaN,NaN
65,y,tvpa_canon,cv,SIV_mask.drawing,2.2,33.8,33.8,33.8,NaN,NaN
66,y,shape_size,cv,SIV_mask.drawing,-0.4,-0.4,-0.4,-0.4,0,NaN
67,y,bool_and_not,cv,or1.tccDebug,or2.tccDebug,JM2_mask.drawing,NaN,NaN,NaN,NaN


In [86]:
lst[1]['122700c_d3t4_siv374d']

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
70,y,StartLayoutAssembler,canon27d3t4lib2_p27_lay,122700c_d3t4_siv374d,a,61.74,63.72,0,0,NaN
71,y,create_rectangle,cv,or2.tccDebug,-24 -24 24 24,NaN,NaN,NaN,NaN,NaN
72,y,chopped_tvpa_canon,cv,JM2_mask.drawing,36,2,9,0.5,NaN,NaN
73,y,chopped_tvpa_canon,cv,SIV_mask.drawing,36,2,9,0.5,NaN,NaN
74,y,shape_size,cv,SIV_mask.drawing,-0.5,-0.5,-0.5,-0.5,0,NaN
75,y,bool_and_not,cv,or1.tccDebug,or2.tccDebug,JM2_mask.drawing,NaN,NaN,NaN,NaN
